In [517]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

In [518]:
# d = range(18202,18962)
match_range = range(18202,18242)

In [519]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [520]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [521]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [522]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [523]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [524]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [525]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [526]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
result_list = []
team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    result = []
    team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    xG_list.append(xG)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [527]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]

#Repeat for all
#Flattening nested lists

In [528]:
col_names = ['match_id','x','y','xG','minute','result','team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,result_list,team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
# df = pd.DataFrame([all_data[0],all_data[1],all_data[2],all_data[3],all_data[4],all_data[5],all_data[6],all_data[7],all_data[8],all_data[9],all_data[10]],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])

In [529]:
df['away_score'] = 0
df['home_score'] = 0

Up next, need to figure out how to add score once we see goal, and keep it there for all of the subsequent rows for that game

In [530]:
df

,match_id,x,y,xG,minute,result,team,h_a,player,home_score,away_score
11,18202,0.8830000305175781,0.5070000076293946,0.18183407187461853,3,BlockedShot,Arsenal,a,Gabriel Jesus,0,0
12,18202,0.935,0.6120000076293945,0.3404459059238434,3,MissedShots,Arsenal,a,Gabriel Martinelli,0,0
13,18202,0.875,0.710999984741211,0.04480058327317238,7,SavedShot,Arsenal,a,Oleksandr Zinchenko,0,0
14,18202,0.8169999694824219,0.5640000152587891,0.04902607202529907,8,BlockedShot,Arsenal,a,Thomas Partey,0,0
15,18202,0.975,0.46900001525878904,0.5384948253631592,19,Goal,Arsenal,a,Gabriel Martinelli,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1040,18241,0.9209999847412109,0.41,0.11410751938819885,91,BlockedShot,Tottenham,a,Djed Spence,0,0
1019,18241,0.8059999847412109,0.725999984741211,0.06215503811836243,94,BlockedShot,Nottingham Forest,h,Emmanuel Dennis,0,0
1020,18241,0.8480000305175781,0.5479999923706055,0.04771732911467552,96,MissedShots,Nottingham Forest,h,Remo Freuler,0,0
1021,18241,0.9069999694824219,0.39799999237060546,0.08542998135089874,96,MissedShots,Nottingham Forest,h,Joe Worrall,0,0


In [531]:
df.loc[df['result'] == 'Goal', 'away_score'] = 10

In [532]:
df

,match_id,x,y,xG,minute,result,team,h_a,player,home_score,away_score
11,18202,0.8830000305175781,0.5070000076293946,0.18183407187461853,3,BlockedShot,Arsenal,a,Gabriel Jesus,0,0
12,18202,0.935,0.6120000076293945,0.3404459059238434,3,MissedShots,Arsenal,a,Gabriel Martinelli,0,0
13,18202,0.875,0.710999984741211,0.04480058327317238,7,SavedShot,Arsenal,a,Oleksandr Zinchenko,0,0
14,18202,0.8169999694824219,0.5640000152587891,0.04902607202529907,8,BlockedShot,Arsenal,a,Thomas Partey,0,0
15,18202,0.975,0.46900001525878904,0.5384948253631592,19,Goal,Arsenal,a,Gabriel Martinelli,0,10
...,...,...,...,...,...,...,...,...,...,...,...
1040,18241,0.9209999847412109,0.41,0.11410751938819885,91,BlockedShot,Tottenham,a,Djed Spence,0,0
1019,18241,0.8059999847412109,0.725999984741211,0.06215503811836243,94,BlockedShot,Nottingham Forest,h,Emmanuel Dennis,0,0
1020,18241,0.8480000305175781,0.5479999923706055,0.04771732911467552,96,MissedShots,Nottingham Forest,h,Remo Freuler,0,0
1021,18241,0.9069999694824219,0.39799999237060546,0.08542998135089874,96,MissedShots,Nottingham Forest,h,Joe Worrall,0,0
